In [1]:
import pandas as pd
import requests
import json
# Assign private constants from a separate file, for API calls to brewerydb and untappd
from api_keys import prodKEY, prodURL, untappd_ID, untappd_SECRET, untappd_URL

### Start by getting all beer styles listed on breweryDB

In [2]:
all_styles = requests.get(prodURL + 'styles?key=' + prodKEY).json()
# parse that JSON response into a readable list
style_list = [(style['id'], style['name']) for style in all_styles['data']]

In [5]:
print(len(style_list))
print(style_list[24:32], style_list[-11:-4])

181
[(25, 'American-Style Pale Ale'), (26, 'Fresh "Wet" Hop Ale'), (27, 'Pale American-Belgo-Style Ale'), (28, 'Dark American-Belgo-Style Ale'), (29, 'American-Style Strong Pale Ale'), (30, 'American-Style India Pale Ale'), (31, 'Imperial or Double India Pale Ale'), (32, 'American-Style Amber/Red Ale')] [(171, 'Juicy or Hazy Pale Ale'), (172, 'Juicy or Hazy India Pale Ale'), (173, 'Juicy or Hazy Imperial or Double India Pale Ale'), (174, 'Contemporary American-Style Pilsener'), (175, 'Classic Australian-Style Pale Ale'), (176, 'Juicy or Hazy Strong Pale Ale'), (177, 'Contemporary Belgian-Style Gueuze Lambic')]


In [6]:
# I'm going to choose these 6 styles from the above list to focus on for this project.
# ('Fresh "Wet" Hop Ale', 'American-Style India Pale Ale', 'Imperial or Double India Pale Ale',
#  'Juicy or Hazy Pale Ale', 'Juicy or Hazy India Pale Ale', 'Juicy or Hazy Imperial or Double India Pale Ale' )
ipas = [26,30,31,171,172,173]

### Using those 6 style ID's, amass the IPA's breweryDB has listed

In [7]:
# The breweryDB API returns beers 50 per page
def getBeerPageByStyle(styleID, page):
    method = 'beers/'
    query = prodURL + method
    params = {'styleId': str(styleID), 'p': str(page), 'key': prodKEY,
              'withSocialAccounts': 'Y', 'withIngredients': 'Y', 'withBreweries': 'Y'}
    response = requests.get(query, params)
    
    # parse the response
    if response:  # response==True for codes 200-400, False otherwise
        remaining_calls = response.headers['X-Ratelimit-Remaining'] # 200 API calls per day is the limit
        return remaining_calls, response.json()
    else: 
        print(f"That GET request with params={params.items()} failed, with code: {response.status_code}")
        print(response.json())
        return 0,0
    

In [8]:
# Sample response for style 30 (American IPA)
calls_left, resp = getBeerPageByStyle(30, 1)

print(f'You have {calls_left} daily calls left.')
# uncomment below to see example response
#resp  

You have 199 daily calls left.


In [10]:
# Show a few beers on that first style 30 page, along with their brewery.
#  (Only going to use the first brewer listed, in the case where there are more than one for the beer.  Cleaner.)
[(beer['breweries'][0]['name'], beer['name']) for beer in resp['data']][:5]

[('Dust Bowl Brewing Company', '"Galactic Wrath" IPA'),
 ('Working Man Brewing Company', '"Ignition" IPA'),
 ("Three Notch'd Brewing Company", '"Roux 40" Red IPA'),
 ('Little Machine Beer', '"Sniff" IPA'),
 ('Four Fathers Brewing, LLC', '#15')]

In [11]:
resp.keys()

dict_keys(['currentPage', 'numberOfPages', 'totalResults', 'data', 'status'])

In [12]:
# Use response['numberOfPages'] to learn how many calls to make to each style
pages = resp['numberOfPages']
pages

199

In [13]:
def growDF(df, styleID, start_page, num_pages=10):  # there's a rate limit like 10/sec on brewerydb
    """Build a pandas DataFrame for each IPA style."""
    for i in range(num_pages):
        limit, beer_data = getBeerPageByStyle(styleID, start_page+i)
        print(f'You have {limit} calls left today.')
        if beer_data:  # This will evaluate to False if the GET method failed
            df = df.append(pd.DataFrame(beer_data['data']))
    return df

In [14]:
# Initialize a DF with page 1 of style 30
style_30_df = pd.DataFrame(resp['data'])

style_30_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
abv                          45 non-null object
available                    21 non-null object
availableId                  21 non-null float64
breweries                    50 non-null object
createDate                   50 non-null object
description                  38 non-null object
foodPairings                 1 non-null object
glass                        12 non-null object
glasswareId                  12 non-null float64
ibu                          30 non-null object
id                           50 non-null object
ingredients                  1 non-null object
isOrganic                    50 non-null object
isRetired                    50 non-null object
labels                       22 non-null object
name                         50 non-null object
nameDisplay                  50 non-null object
originalGravity              8 non-null object
servingTemperature           8 non

Well, that stinks that only 1 of the first 50 IPA's has ingredients listed, but at least 38 of them have descriptions, which maybe can be analyzed for keywords like ingredients.

In [16]:
# concatenate vertically pages 2-199
##  NOTE THAT THIS WILL USE ALL YOUR CALLS UP FOR A DAY, FOR STYLE 30 ##
style_30_df = growDF(style_30_df, styleID=30, start_page=2, num_pages=198)

In [17]:
len(style_30_df)

In [ ]:
style_30_df.to_pickle('style_30_df.pkl')

### So that's it for the largest style, #30, and that's it for API calls for the day.
### Here's the encapsulated routine for the other 5 styles:

In [48]:
ipas.remove(30)

ipas

[26, 31, 171, 172, 173]

In [51]:
for ipa in ipas:
    # initialize a pd DF from the first page of the style
    calls_left, resp = getBeerPageByStyle(ipa, page=1)
    print(f'You have {calls_left} daily calls left.')
    df = pd.DataFrame(resp['data'])
    # see how many total pages to query
    pages = resp['numberOfPages']
    df = growDF(df, styleID=ipa, start_page=2, num_pages=pages-1)
    # pickle the resulting df
    print(f'Pickling {len(df)} style {ipa} IPAs to "style_{ipa}_df.pkl"')
    df.to_pickle('style_' + str(ipa) + '_df.pkl')
    

You have 197 daily calls left.
You have 196 calls left today.
You have 195 calls left today.
You have 194 calls left today.
You have 193 calls left today.
You have 192 calls left today.
You have 191 calls left today.
You have 190 calls left today.
You have 189 calls left today.
You have 188 calls left today.
You have 187 calls left today.
You have 186 calls left today.
You have 185 calls left today.
You have 184 calls left today.
You have 183 calls left today.
You have 182 calls left today.
You have 181 calls left today.
You have 180 calls left today.
You have 179 calls left today.
You have 178 calls left today.
You have 177 calls left today.
You have 176 calls left today.
You have 175 calls left today.
You have 174 calls left today.
You have 173 calls left today.
You have 172 calls left today.
You have 171 calls left today.
You have 170 calls left today.
You have 169 calls left today.
You have 168 calls left today.
You have 167 calls left today.
You have 166 calls left today.
You have